In [1]:
import numpy as np
import pickle
import csv

In [2]:
folder = 'm20_10'

In [3]:
_d_result = pickle.load(open('../data/'+folder+'/ranking_matrix_director.pkl', 'rb'))
_t_result = pickle.load(open('../data/'+folder+'/ranking_matrix_type.pkl', 'rb'))
_g_result = pickle.load(open('../data/'+folder+'/ranking_matrix_genre.pkl', 'rb'))
_c_result = pickle.load(open('../data/'+folder+'/ranking_matrix_cast.pkl', 'rb'))

In [4]:
d_shape = _d_result.shape
t_shape = _t_result.shape
g_shape = _g_result.shape
c_shape = _c_result.shape

In [5]:
id_2_genre = pickle.load(open('../data/'+folder+'/id_2_genre.pkl', 'rb'))
id_2_director = pickle.load(open('../data/'+folder+'/id_2_director.pkl', 'rb'))
id_2_cast = pickle.load(open('../data/'+folder+'/id_2_cast.pkl', 'rb'))
genre_2_id = pickle.load(open('../data/'+folder+'/genre_2_id.pkl', 'rb'))
director_2_id = pickle.load(open('../data/'+folder+'/director_2_id.pkl', 'rb'))
cast_2_id = pickle.load(open('../data/'+folder+'/cast_2_id.pkl', 'rb'))

In [6]:
def load_data(user_name, movie_name, rating_name, last_same):
    user = pickle.load(open('../data/'+folder+'/'+user_name+last_same+'.pkl', 'rb'))
    movie = pickle.load(open('../data/'+folder+'/'+movie_name+last_same+'.pkl', 'rb'))
    rating = pickle.load(open('../data/'+folder+'/'+rating_name+last_same+'.pkl', 'rb'))
    return user, movie, rating

#load training data
d_user, d_movie, d_rating = load_data("train_user", "train_movie", "train_rating", '_d')
t_user, t_movie, t_rating = load_data("train_user", "train_movie", "train_rating", '_t')
g_user, g_movie, g_rating = load_data("train_user", "train_movie", "train_rating", '_g')   
c_user, c_movie, c_rating = load_data("train_user", "train_movie", "train_rating", '_c')

#load validation data
d_user_v, d_movie_v, d_rating_v = load_data("train_user", "train_movie", "train_rating", '_d_v')
t_user_v, t_movie_v, t_rating_v = load_data("train_user", "train_movie", "train_rating", '_t_v')
g_user_v, g_movie_v, g_rating_v = load_data("train_user", "train_movie", "train_rating", '_g_v')        
c_user_v, c_movie_v, c_rating_v = load_data("train_user", "train_movie", "train_rating", '_c_v')


"""
d_user += d_user_v
d_movie += d_movie_v
d_rating += d_rating_v
t_user += t_user_v
t_movie += t_movie_v
t_rating += t_rating_v
g_user += g_user_v
g_movie += g_movie_v
g_rating += g_rating_v
c_user += c_user_v
c_movie += c_movie_v
c_rating += c_rating_v
"""

'\nd_user += d_user_v\nd_movie += d_movie_v\nd_rating += d_rating_v\nt_user += t_user_v\nt_movie += t_movie_v\nt_rating += t_rating_v\ng_user += g_user_v\ng_movie += g_movie_v\ng_rating += g_rating_v\nc_user += c_user_v\nc_movie += c_movie_v\nc_rating += c_rating_v\n'

# Generate dict

In [7]:
def generate_dict(user, movie, rating):
    l = len(user)
    r_dict = {}
    for i in range(l):
        tup = (user[i], movie[i])
        if  movie[i] not in r_dict:
            r_dict[movie[i]] = {}
        
        if user[i] in r_dict[movie[i]]:
            print(user[i], movie[i], 'wrong')
        else:
            r_dict[movie[i]][user[i]] = rating[i]
            
            
    return r_dict

In [8]:
d_dict = generate_dict(d_user, d_movie, d_rating)
t_dict = generate_dict(t_user, t_movie, t_rating)
g_dict = generate_dict(g_user, g_movie, g_rating)
c_dict = generate_dict(c_user, c_movie, c_rating)

In [9]:
def gen_unknown_dict(ressult_mat, f_dict):
    unknown_dict = {}
    for i in range(len(ressult_mat[0])):  #feature
        unknown_dict[i] = {}
        for j in range(len(ressult_mat)): #user
            if i not in f_dict or j not in f_dict[i]:
                unknown_dict[i][j] = ressult_mat[j, i]
    
    return unknown_dict

In [10]:
d_dict_unknown = gen_unknown_dict(_d_result, d_dict)
t_dict_unknown = gen_unknown_dict(_t_result, t_dict)
g_dict_unknown = gen_unknown_dict(_g_result, g_dict)
c_dict_unknown = gen_unknown_dict(_c_result, c_dict)

In [11]:
def total_mat_rate(tot_mat):
    mean_rate = []
    for i in range(len(tot_mat[0])):
        mean_rate.append(np.mean(tot_mat[:,i]))
    return mean_rate

In [12]:
def get_mean_rating(_dict, shape):
    mean_rate = [0 for i in range(shape[1])]
    for i in range(shape[1]):
        if i not in _dict:
            mean_rate[i] = 0
        else:
            arr = []
            for key in _dict[i]:
                arr.append(float(_dict[i][key]))
            mean_rate[i] = np.mean(arr)
    
    return mean_rate
        
        

In [13]:
def analysis(_dict, tot_result, _shape, id_2_feature, file_name):
    csvFile = open('../data/' +folder+ '/' + file_name + '.csv', "w")
    writer = csv.writer(csvFile)
    
    _mean = get_mean_rating(_dict, _shape)
    _mean_unknown = total_mat_rate(tot_result)
    _rating = np.argsort(_mean)[::-1]
    _rating_unknown = np.argsort(_mean_unknown)[::-1]
    ind = 0
    for i in _rating_unknown:
        name = id_2_feature[i] if i in id_2_feature else ''
        print(ind, '(', np.where(_rating==i)[0][0], ')', _mean_unknown[i], '(', _mean[i], ')', name)
        writer.writerow([ind, np.where(_rating==i)[0][0], _mean_unknown[i], _mean[i], name])
        ind += 1

    

In [14]:
analysis(g_dict, _g_result, g_shape, id_2_genre, 'genre_rank')

0 ( 6 ) 4.11493 ( 3.62804007444 ) Drama
1 ( 3 ) 4.04574 ( 3.69989697882 ) War
2 ( 4 ) 4.03308 ( 3.66569241529 ) Biography
3 ( 7 ) 4.02935 ( 3.57612728868 ) Crime
4 ( 5 ) 3.94274 ( 3.64847154645 ) History
5 ( 0 ) 3.93706 ( 3.92746901923 ) Film-Noir
6 ( 1 ) 3.92751 ( 3.83881029017 ) Short
7 ( 8 ) 3.88245 ( 3.48705633477 ) Mystery
8 ( 2 ) 3.79435 ( 3.70359147947 ) Documentary
9 ( 13 ) 3.79235 ( 3.38346565068 ) Thriller
10 ( 14 ) 3.7668 ( 3.37720273676 ) Fantasy
11 ( 10 ) 3.75956 ( 3.43587504901 ) Animation
12 ( 17 ) 3.71619 ( 3.33012960016 ) Sci-Fi
13 ( 15 ) 3.71402 ( 3.35058574646 ) Adventure
14 ( 16 ) 3.70438 ( 3.33926860251 ) Romance
15 ( 18 ) 3.69421 ( 3.32503496399 ) Comedy
16 ( 11 ) 3.67264 ( 3.41485266328 ) Music
17 ( 12 ) 3.67108 ( 3.40379310749 ) Musical
18 ( 19 ) 3.62331 ( 3.24994734126 ) Action
19 ( 9 ) 3.56551 ( 3.48005487963 ) Western
20 ( 22 ) 3.5078 ( 3.19170483167 ) Family
21 ( 21 ) 3.47666 ( 3.20488183091 ) Horror
22 ( 20 ) 3.35398 ( 3.21302405268 ) Sport
23 ( 23 ) 2.5804

In [15]:
analysis(d_dict, _d_result, d_shape, id_2_director, 'director_rank')

0 ( 16 ) 4.14174 ( 4.08262711864 ) Lana Wachowski
1 ( 15 ) 4.14172 ( 4.08262711864 ) Lilly Wachowski
2 ( 12 ) 4.12196 ( 4.10394648549 ) Akira Kurosawa
3 ( 18 ) 4.12172 ( 4.07049608355 ) Clyde Bruckman
4 ( 1583 ) 4.11696 ( 2.625 ) Robert Bierman
5 ( 10 ) 4.11141 ( 4.10890099909 ) Frank Darabont
6 ( 68 ) 4.08626 ( 3.95627062706 ) Mathieu Kassovitz
7 ( 26 ) 4.08625 ( 4.01744186047 ) Thomas Vinterberg
8 ( 20 ) 4.077 ( 4.05396777136 ) Tony Kaye
9 ( 71 ) 4.06886 ( 3.95270890725 ) Emir Kusturica
10 ( 21 ) 4.06849 ( 4.05280337044 ) Michael Curtiz
11 ( 33 ) 4.05748 ( 4.00272486249 ) Quentin Tarantino
12 ( 60 ) 4.05107 ( 3.97934076137 ) Ingmar Bergman
13 ( 22 ) 4.0497 ( 4.05191768007 ) Carol Reed
14 ( 56 ) 4.04902 ( 3.98685211185 ) Buster Keaton
15 ( 32 ) 4.04694 ( 4.00698474103 ) Sam Mendes
16 ( 61 ) 4.04428 ( 3.97674418605 ) Ron Fricke
17 ( 27 ) 4.0437 ( 4.01724137931 ) Hayao Miyazaki
18 ( 25 ) 4.03282 ( 4.01809392265 ) Robert Mulligan
19 ( 55 ) 4.03281 ( 3.98980537535 ) Roberto Benigni
20 ( 5

334 ( 657 ) 3.56952 ( 3.41379310345 ) C_dric Klapisch
335 ( 612 ) 3.56909 ( 3.44265934827 ) Allison Anders
336 ( 565 ) 3.56868 ( 3.47951509035 ) T. Hee
337 ( 600 ) 3.56812 ( 3.4511439581 ) Robert Wise
338 ( 582 ) 3.56799 ( 3.46566523605 ) Roger Nygard
339 ( 494 ) 3.56792 ( 3.51776649746 ) Julie Taymor
340 ( 655 ) 3.56583 ( 3.41468741469 ) Carl Reiner
341 ( 495 ) 3.56496 ( 3.51744186047 ) Mark Rydell
342 ( 518 ) 3.56467 ( 3.5 ) Seth Zvi Rosenfeld
343 ( 577 ) 3.56366 ( 3.47111553785 ) George Dunning
344 ( 420 ) 3.56211 ( 3.57831325301 ) James Clavell
345 ( 578 ) 3.55985 ( 3.47051597052 ) Peter Brook
346 ( 500 ) 3.55964 ( 3.5112454792 ) Robert Redford
347 ( 546 ) 3.5596 ( 3.49295582 ) Guy Hamilton
348 ( 585 ) 3.55923 ( 3.46533613445 ) Jamie Uys
349 ( 539 ) 3.5587 ( 3.49844622768 ) Andrew Davis
350 ( 508 ) 3.55784 ( 3.50317568938 ) James Cameron
351 ( 306 ) 3.55711 ( 3.67420091324 ) Joseph Sargent
352 ( 638 ) 3.55676 ( 3.42708572472 ) John Frankenheimer
353 ( 683 ) 3.55553 ( 3.39613821138 

566 ( 955 ) 3.32535 ( 3.19286449148 ) Andy Tennant
567 ( 905 ) 3.32406 ( 3.2264957265 ) Tom Hanks
568 ( 900 ) 3.32269 ( 3.23053892216 ) Lewis John Carlino
569 ( 999 ) 3.32218 ( 3.15927128427 ) Tamra Davis
570 ( 897 ) 3.32218 ( 3.23553377534 ) Phillip Noyce
571 ( 974 ) 3.32067 ( 3.17276422764 ) Steve Hickner
572 ( 622 ) 3.3203 ( 3.43548387097 ) Al Pacino
573 ( 970 ) 3.31924 ( 3.17518518519 ) Abel Ferrara
574 ( 954 ) 3.31914 ( 3.19334532374 ) Julian Harris
575 ( 915 ) 3.31789 ( 3.21640574038 ) Ronald Neame
576 ( 953 ) 3.31618 ( 3.19334532374 ) Pino Van Lamsweerde
577 ( 977 ) 3.31241 ( 3.16929133858 ) Steve Kloves
578 ( 947 ) 3.31216 ( 3.19334532374 ) Jack Stokes
579 ( 1004 ) 3.31167 ( 3.15015360983 ) Gary Fleder
580 ( 1019 ) 3.30939 ( 3.13957055215 ) Mel Welles
581 ( 975 ) 3.30926 ( 3.17153419867 ) Jay Roach
582 ( 496 ) 3.30924 ( 3.51709401709 ) Istvn Szab
583 ( 978 ) 3.30826 ( 3.1690984888 ) Brett Ratner
584 ( 950 ) 3.30601 ( 3.19334532374 ) John Halas
585 ( 959 ) 3.30397 ( 3.18711038

778 ( 1317 ) 3.07471 ( 2.89338471419 ) Don Taylor
779 ( 1240 ) 3.07406 ( 2.97395833333 ) Tim Hill
780 ( 1198 ) 3.07264 ( 3.0 ) Bradley Battersby
781 ( 1257 ) 3.06597 ( 2.95446348351 ) Garry Marshall
782 ( 1253 ) 3.06501 ( 2.96073298429 ) Kirk Wong
783 ( 1299 ) 3.06484 ( 2.91415662651 ) Phil Nibbelink
784 ( 1314 ) 3.0639 ( 2.89602672293 ) Roger Spottiswoode
785 ( 310 ) 3.06199 ( 3.66923076923 ) Yang Zhang
786 ( 1214 ) 3.05957 ( 3.0 ) Robert F. Hill
787 ( 466 ) 3.05923 ( 3.53571428571 ) Eugene Corr
788 ( 113 ) 3.05923 ( 3.88425925926 ) John Cassavetes
789 ( 1321 ) 3.05768 ( 2.88801115242 ) Stephen Surjik
790 ( 1215 ) 3.05677 ( 3.0 ) Doug Block
791 ( 1355 ) 3.05643 ( 2.86879795396 ) Hugh Wilson
792 ( 996 ) 3.0555 ( 3.16383495146 ) Richard Quine
793 ( 641 ) 3.0535 ( 3.42647058824 ) Lisa Cholodenko
794 ( 1139 ) 3.05313 ( 3.04838709677 ) Irvin S. Yeaworth Jr.
795 ( 1338 ) 3.0509 ( 2.87556306306 ) Barbet Schroeder
796 ( 1351 ) 3.05028 ( 2.87081005587 ) Brian Gibson
797 ( 1288 ) 3.04995 ( 2.92

1004 ( 1531 ) 2.79602 ( 2.69494047619 ) Jerry Lewis
1005 ( 1552 ) 2.79531 ( 2.67275280899 ) Jocelyn Moorhouse
1006 ( 1631 ) 2.7944 ( 2.56592039801 ) Ringo Lam
1007 ( 932 ) 2.78938 ( 3.2 ) Irving Rapper
1008 ( 1457 ) 2.7884 ( 2.77597402597 ) Stephen Gyllenhaal
1009 ( 276 ) 2.78828 ( 3.7 ) Mweze Ngangura
1010 ( 1395 ) 2.78703 ( 2.83333333333 ) Jean Delannoy
1011 ( 1578 ) 2.78486 ( 2.63171247357 ) Irwin Winkler
1012 ( 1601 ) 2.78322 ( 2.61101836394 ) Susan Seidelman
1013 ( 361 ) 2.7829 ( 3.63157894737 ) Anthony Harvey
1014 ( 827 ) 2.78269 ( 3.28947368421 ) Sarah Kelly
1015 ( 401 ) 2.78214 ( 3.59090909091 ) Hiroshi Inagaki
1016 ( 1549 ) 2.78129 ( 2.67697594502 ) Robert Longo
1017 ( 94 ) 2.78001 ( 3.91666666667 ) S.R. Bindler
1018 ( 427 ) 2.77957 ( 3.57142857143 ) Nanette Burstein
1019 ( 1545 ) 2.77783 ( 2.67934782609 ) Bruce Paltrow
1020 ( 1588 ) 2.77719 ( 2.62024916753 ) Frank Marshall
1021 ( 593 ) 2.77679 ( 3.46031746032 ) Simon Shore
1022 ( 1280 ) 2.77653 ( 2.93055555556 ) Anton Leader


1226 ( 1749 ) 2.47646 ( 2.40338010204 ) Edward D. Wood Jr.
1227 ( 1732 ) 2.47289 ( 2.43913043478 ) Pat Proft
1228 ( 1184 ) 2.4725 ( 3.0 ) Maya Angelou
1229 ( 1802 ) 2.47249 ( 2.3307310149 ) Jim Gillespie
1230 ( 1723 ) 2.47187 ( 2.45 ) Norton Virgien
1231 ( 1346 ) 2.47139 ( 2.875 ) Armando Crispino
1232 ( 1804 ) 2.47051 ( 2.32203947368 ) Peter MacDonald
1233 ( 1292 ) 2.46604 ( 2.92105263158 ) Bruce Malmuth
1234 ( 1646 ) 2.46558 ( 2.52884615385 ) Catherine Breillat
1235 ( 1745 ) 2.46507 ( 2.41346153846 ) Kevin Rodney Sullivan
1236 ( 322 ) 2.46505 ( 3.66346153846 ) Marco Ferreri
1237 ( 1011 ) 2.46187 ( 3.14583333333 ) Charles Sturridge
1238 ( 1785 ) 2.46077 ( 2.34637188209 ) Peter Medak
1239 ( 1776 ) 2.45861 ( 2.35714285714 ) Leslie Greif
1240 ( 1780 ) 2.45727 ( 2.35558889722 ) Joe Pytka
1241 ( 1690 ) 2.45556 ( 2.5 ) Gray Hofmeyr
1242 ( 383 ) 2.45532 ( 3.60426731079 ) Anthony Mann
1243 ( 1852 ) 2.45358 ( 2.25 ) Larry Karaszewski
1244 ( 1831 ) 2.45138 ( 2.26930501931 ) Alan Shapiro
1245 ( 

1475 ( 594 ) 2.12724 ( 3.45833333333 ) ke Sandgren
1476 ( 1773 ) 2.1224 ( 2.36666666667 ) Richard C. Sarafian
1477 ( 1558 ) 2.1211 ( 2.66666666667 ) Terry O. Morse
1478 ( 2184 ) 2.11879 ( 0 ) Mel Chionglo
1479 ( 1974 ) 2.11462 ( 1.93180031447 ) Bryan Spicer
1480 ( 775 ) 2.11445 ( 3.33333333333 ) Bill Milling
1481 ( 1404 ) 2.11407 ( 2.825 ) Richard Shepard
1482 ( 23 ) 2.11278 ( 4.02941176471 ) Ren_ Cl_ment
1483 ( 744 ) 2.11132 ( 3.36 ) Douglas Keeve
1484 ( 748 ) 2.11132 ( 3.35714285714 ) Robert Z. Leonard
1485 ( 1560 ) 2.10995 ( 2.66666666667 ) Kiefer Sutherland
1486 ( 568 ) 2.10945 ( 3.47727272727 ) Liliana Cavani
1487 ( 1924 ) 2.10937 ( 2.09401709402 ) Chris Roberts
1488 ( 766 ) 2.10862 ( 3.34375 ) Alex Sichel
1489 ( 1573 ) 2.10583 ( 2.640625 ) R.J. Kizer
1490 ( 1909 ) 2.10483 ( 2.12345679012 ) James Orr
1491 ( 1720 ) 2.10343 ( 2.45081967213 ) Gregg Champion
1492 ( 1472 ) 2.10233 ( 2.76388888889 ) Erle C. Kenton
1493 ( 555 ) 2.10217 ( 3.49122807018 ) Max F_rberb_ck
1494 ( 2088 ) 2.10

1692 ( 929 ) 1.80906 ( 3.20588235294 ) Raoul Ruiz
1693 ( 2058 ) 1.80773 ( 1.55913978495 ) Richard Martin
1694 ( 1347 ) 1.80649 ( 2.875 ) Chris Browne
1695 ( 1686 ) 1.80131 ( 2.5 ) Anna Campion
1696 ( 1767 ) 1.79972 ( 2.375 ) Matthew Harrison
1697 ( 2092 ) 1.79932 ( 1.25 ) Michael Verhoeven
1698 ( 228 ) 1.79595 ( 3.75 ) Tom McNamara
1699 ( 2004 ) 1.7929 ( 1.79285714286 ) David Price
1700 ( 1180 ) 1.79135 ( 3.0 ) Busby Berkeley
1701 ( 45 ) 1.78967 ( 4.0 ) Richard Gordon
1702 ( 1772 ) 1.78732 ( 2.36734693878 ) Zalman King
1703 ( 1839 ) 1.78621 ( 2.25 ) Arthur Joff_
1704 ( 1010 ) 1.78602 ( 3.14705882353 ) Alan Taylor
1705 ( 993 ) 1.78585 ( 3.16666666667 ) Scott Ziehl
1706 ( 2055 ) 1.78483 ( 1.56432748538 ) Farhad Mann
1707 ( 1866 ) 1.78411 ( 2.21428571429 ) Wesley Strick
1708 ( 1162 ) 1.78054 ( 3.02380952381 ) Roy William Neill
1709 ( 1129 ) 1.78026 ( 3.05555555556 ) Shane Meadows
1710 ( 476 ) 1.7798 ( 3.52777777778 ) Gavin O'Connor
1711 ( 1737 ) 1.77778 ( 2.42857142857 ) Richard Schenkman

1911 ( 1951 ) 1.45088 ( 2.0 ) Ildik Enyedi
1912 ( 1946 ) 1.44898 ( 2.0 ) Milton Moses Ginsberg
1913 ( 1942 ) 1.44516 ( 2.0 ) Bruce Kessler
1914 ( 1490 ) 1.44322 ( 2.75 ) Radley Metzger
1915 ( 9 ) 1.44305 ( 4.125 ) Maria Ripoll
1916 ( 1508 ) 1.44264 ( 2.72727272727 ) Carl Prechezer
1917 ( 1422 ) 1.43974 ( 2.80952380952 ) Charlie Peters
1918 ( 2054 ) 1.43569 ( 1.59119496855 ) Sam Pillsbury
1919 ( 2109 ) 1.43189 ( 0.5 ) Gabriel Barylli
1920 ( 2151 ) 1.43145 ( 0 ) Michael Goetz
1921 ( 1193 ) 1.43109 ( 3.0 ) Yuan Zhang
1922 ( 1210 ) 1.42544 ( 3.0 ) Christopher C. Frieri
1923 ( 1189 ) 1.42444 ( 3.0 ) Hugo Van Lawick
1924 ( 517 ) 1.4226 ( 3.5 ) John S. Robertson
1925 ( 2013 ) 1.41609 ( 1.76923076923 ) Deborah Brock
1926 ( 36 ) 1.41566 ( 4.0 ) Clare Peploe
1927 ( 1202 ) 1.41405 ( 3.0 ) Victoria Foyt
1928 ( 1434 ) 1.4133 ( 2.8 ) Klaus Kinski
1929 ( 2033 ) 1.41179 ( 1.69117647059 ) Mamoru Hosoda
1930 ( 2035 ) 1.40825 ( 1.68181818182 ) Rafal Zielinski
1931 ( 759 ) 1.40514 ( 3.35 ) Michael Rymer


2113 ( 1680 ) 0.880954 ( 2.5 ) Paul Murray
2114 ( 1569 ) 0.880106 ( 2.64285714286 ) Margarethe von Trotta
2115 ( 37 ) 0.853918 ( 4.0 ) Kristine Peterson
2116 ( 70 ) 0.851595 ( 3.95454545455 ) David Moreton
2117 ( 995 ) 0.849326 ( 3.16666666667 ) Theodoros Angelopoulos
2118 ( 1724 ) 0.838571 ( 2.45 ) Corey Allen
2119 ( 2007 ) 0.835611 ( 1.78947368421 ) Peter R. Simpson
2120 ( 1204 ) 0.828656 ( 3.0 ) Hilary Brougher
2121 ( 1517 ) 0.828574 ( 2.71428571429 ) Ash Baron-Cohen
2122 ( 1922 ) 0.824657 ( 2.10294117647 ) Gary Halvorson
2123 ( 274 ) 0.821709 ( 3.7 ) Benoît Lamy
2124 ( 878 ) 0.807384 ( 3.25 ) C_dric Kahn
2125 ( 1319 ) 0.802362 ( 2.88888888889 ) Theodore Witcher
2126 ( 1873 ) 0.799203 ( 2.1875 ) Stephen Carpenter
2127 ( 1701 ) 0.798045 ( 2.5 ) Adam Bernstein
2128 ( 1207 ) 0.794203 ( 3.0 ) Rolf de Heer
2129 ( 1479 ) 0.777851 ( 2.75 ) Franois Ozon
2130 ( 2021 ) 0.772387 ( 1.75 ) John O'Hagan
2131 ( 713 ) 0.770608 ( 3.375 ) Peter Duffell
2132 ( 234 ) 0.768155 ( 3.75 ) Daphna Kastner
2

In [16]:
analysis(c_dict, _c_result, c_shape, id_2_cast, 'cast_rank')

0 ( 1603 ) 4.42139 ( 3.35869565217 ) Shirley Henderson
1 ( 14 ) 4.28956 ( 4.23008175791 ) Amanda Plummer
2 ( 9 ) 4.27712 ( 4.26021433356 ) Takashi Shimura
3 ( 1418 ) 4.18871 ( 3.42905405405 ) Diego Abatantuono
4 ( 17 ) 4.17755 ( 4.15545697768 ) Toshir» Mifune
5 ( 39 ) 4.15926 ( 4.10118137301 ) Tim Roth
6 ( 15 ) 4.14179 ( 4.18962203716 ) John Fiedler
7 ( 21 ) 4.13531 ( 4.14002828854 ) Eli Wallach
8 ( 75 ) 4.11907 ( 4.04112339298 ) Stephen Baldwin
9 ( 87 ) 4.11112 ( 4.02632236576 ) Claudia Cardinale
10 ( 32 ) 4.10972 ( 4.11479591837 ) Ralph Meeker
11 ( 16 ) 4.10438 ( 4.16687817259 ) Gloria Swanson
12 ( 24 ) 4.10262 ( 4.13700692562 ) Michael Berryman
13 ( 25 ) 4.10234 ( 4.13700692562 ) Peter Brocco
14 ( 58 ) 4.09876 ( 4.06457466504 ) Edward Norton
15 ( 60 ) 4.09459 ( 4.06264595659 ) George C. Scott
16 ( 20 ) 4.09027 ( 4.14254859611 ) Alida Valli
17 ( 61 ) 4.09018 ( 4.05867106503 ) Andrew Kevin Walker
18 ( 3536 ) 4.08912 ( 2.1875 ) David Allen Brooks
19 ( 94 ) 4.079 ( 4.02272727273 ) Gunna

345 ( 379 ) 3.76113 ( 3.8397260274 ) Stephen J. Lang
346 ( 106 ) 3.76036 ( 4.01207729469 ) Nobuko Miyamoto
347 ( 636 ) 3.7598 ( 3.72208207198 ) Derek Jacobi
348 ( 690 ) 3.75965 ( 3.69384019494 ) Mia Farrow
349 ( 50 ) 3.7588 ( 4.07657657658 ) Srdjan 'Zika' Todorovic
350 ( 256 ) 3.75837 ( 3.91078838174 ) Tom Schimmels
351 ( 54 ) 3.758 ( 4.07070707071 ) Jane Greer
352 ( 638 ) 3.75615 ( 3.72182365193 ) Michael Biehn
353 ( 657 ) 3.75472 ( 3.71116255144 ) Cate Blanchett
354 ( 620 ) 3.75418 ( 3.72896737797 ) Ray Milland
355 ( 1114 ) 3.75348 ( 3.5332278481 ) Richard Bohringer
356 ( 415 ) 3.74968 ( 3.82352941176 ) Louis Wolheim
357 ( 623 ) 3.74913 ( 3.72875 ) Verna Bloom
358 ( 881 ) 3.74837 ( 3.62743957895 ) Jean Reno
359 ( 670 ) 3.74826 ( 3.70404369642 ) Frances McDormand
360 ( 531 ) 3.74766 ( 3.76823895677 ) Sidney Poitier
361 ( 728 ) 3.74718 ( 3.67564655172 ) John Lennon
362 ( 827 ) 3.74563 ( 3.64519396333 ) Chloï Sevigny
363 ( 333 ) 3.74489 ( 3.86495176849 ) Sam Berry
364 ( 591 ) 3.74209 ( 

584 ( 856 ) 3.61261 ( 3.63691588785 ) Ron Howard
585 ( 1316 ) 3.61108 ( 3.47028317419 ) Matt Stone
586 ( 1140 ) 3.60928 ( 3.52383053839 ) Sandra Prinsloo
587 ( 1235 ) 3.60865 ( 3.4990300679 ) Richard Beymer
588 ( 836 ) 3.60771 ( 3.64304802382 ) Fred Astaire
589 ( 1125 ) 3.60705 ( 3.52855832413 ) Judy Davis
590 ( 1157 ) 3.60702 ( 3.51853932584 ) Dianne Wiest
591 ( 479 ) 3.60663 ( 3.79372197309 ) Masatoshi Nagase
592 ( 1139 ) 3.60658 ( 3.52383053839 ) Marius Weyers
593 ( 1172 ) 3.60648 ( 3.50833333333 ) James Cosmo
594 ( 892 ) 3.60621 ( 3.62386363636 ) Leonard Whiting
595 ( 1168 ) 3.60493 ( 3.51140692458 ) Cameron Diaz
596 ( 1269 ) 3.60476 ( 3.48578409596 ) Brandon Lee
597 ( 1006 ) 3.60396 ( 3.57863427302 ) Bill Murray
598 ( 1059 ) 3.60285 ( 3.55561555076 ) Bernadette Peters
599 ( 1445 ) 3.60269 ( 3.42295081967 ) Sarah Henderson
600 ( 822 ) 3.60145 ( 3.64819700967 ) Telly Savalas
601 ( 1019 ) 3.60134 ( 3.5715674054 ) David Niven
602 ( 871 ) 3.60121 ( 3.63002680965 ) Estelle Harris
603 ( 

782 ( 283 ) 3.50823 ( 3.890625 ) Paul Scofield
783 ( 1501 ) 3.50653 ( 3.39909160982 ) Kathy Bates
784 ( 883 ) 3.50652 ( 3.62653061224 ) Rachel Roberts
785 ( 1536 ) 3.50628 ( 3.3825570301 ) Piper Laurie
786 ( 1455 ) 3.50552 ( 3.41743725231 ) Debbie Reynolds
787 ( 1473 ) 3.50362 ( 3.40971625107 ) Eleanor Audley
788 ( 1359 ) 3.50301 ( 3.45452530807 ) Alec Baldwin
789 ( 1534 ) 3.5028 ( 3.38468992248 ) Mel Brooks
790 ( 1414 ) 3.50136 ( 3.43072435586 ) Roy Atwell
791 ( 1472 ) 3.5013 ( 3.40978002378 ) Phil Harris
792 ( 1339 ) 3.50073 ( 3.45986624858 ) Christopher Lloyd
793 ( 1417 ) 3.50022 ( 3.42915673158 ) Tim Allen
794 ( 1904 ) 3.49963 ( 3.23661800487 ) Hilary Swank
795 ( 1474 ) 3.49955 ( 3.40971625107 ) Ilene Woods
796 ( 1367 ) 3.49863 ( 3.450169054 ) Campbell Scott
797 ( 1533 ) 3.49853 ( 3.38513589896 ) Nick Nolte
798 ( 1626 ) 3.49792 ( 3.34756210705 ) Miranda Richardson
799 ( 867 ) 3.49776 ( 3.63177387914 ) Danny Kaye
800 ( 1296 ) 3.49736 ( 3.47768269643 ) Dorothy McGuire
801 ( 1495 ) 3.

1005 ( 1983 ) 3.36168 ( 3.19912898806 ) Hal Holbrook
1006 ( 1825 ) 3.36138 ( 3.26217228464 ) Diana Rigg
1007 ( 1764 ) 3.3609 ( 3.28959890312 ) Michael Keaton
1008 ( 1789 ) 3.35913 ( 3.28096249115 ) Carole Bouquet
1009 ( 1824 ) 3.35903 ( 3.26249822064 ) Cuba Gooding Jr.
1010 ( 1801 ) 3.35768 ( 3.27610407395 ) Sean Bean
1011 ( 1770 ) 3.35752 ( 3.28708723509 ) Kelly McGillis
1012 ( 1972 ) 3.35698 ( 3.20042553962 ) James Earl Jones
1013 ( 1963 ) 3.35658 ( 3.20593080725 ) Nehemiah Persoff
1014 ( 1914 ) 3.35495 ( 3.23090277778 ) Sonny Bono
1015 ( 1907 ) 3.35234 ( 3.2339475594 ) Leonardo DiCaprio
1016 ( 1362 ) 3.35187 ( 3.45374449339 ) Raquel Welch
1017 ( 1773 ) 3.35117 ( 3.28694430305 ) Nicolas Cage
1018 ( 919 ) 3.35066 ( 3.61519607843 ) Leslie Banks
1019 ( 530 ) 3.3506 ( 3.76829268293 ) Kenneth More
1020 ( 1964 ) 3.35033 ( 3.20531886024 ) Matthew McConaughey
1021 ( 1913 ) 3.34947 ( 3.23105360444 ) J. Pat O'Malley
1022 ( 1747 ) 3.34898 ( 3.29754473575 ) Mickey Rooney
1023 ( 900 ) 3.34871 ( 3

...

2894 ( 3653 ) 1.87553 ( 2.0 ) Lysette Anthony
2895 ( 1909 ) 1.87523 ( 3.23287671233 ) Antnio Assuno
2896 ( 3454 ) 1.87467 ( 2.26923076923 ) Kelly Wolf
2897 ( 3083 ) 1.87423 ( 2.6 ) Juliet Aubrey
2898 ( 2358 ) 1.8742 ( 3.03846153846 ) Keri Jo Chapman
2899 ( 3990 ) 1.87363 ( 0 ) Bruce Friedrich
2900 ( 201 ) 1.8716 ( 3.94117647059 ) Giovanna Bozzolo
2901 ( 3998 ) 1.87082 ( 0 ) Clair D. Alexander
2902 ( 1461 ) 1.87038 ( 3.41397849462 ) Toni Collette
2903 ( 3880 ) 1.87024 ( 1.0 ) Filberto Ascencio
2904 ( 879 ) 1.86983 ( 3.62820512821 ) Anouk Aim_e
2905 ( 955 ) 1.86856 ( 3.59745762712 ) Giancarlo Giannini
2906 ( 2459 ) 1.86806 ( 3.0 ) Gordon Oas-Heim
2907 ( 3237 ) 1.86781 ( 2.5 ) Sophia Adella Luke
2908 ( 1876 ) 1.86705 ( 3.25 ) Jeanie Drynan
2909 ( 2469 ) 1.86576 ( 3.0 ) Samantha Morton
2910 ( 2653 ) 1.86573 ( 2.88095238095 ) Asia Argento
2911 ( 617 ) 1.8634 ( 3.73076923077 ) Julia Migenes
2912 ( 3967 ) 1.86285 ( 0 ) Kathleen Chalfant
2913 ( 3502 ) 1.86157 ( 2.22857142857 ) Andrew Lowery

3193 ( 2645 ) 1.62972 ( 2.88636363636 ) Blair Brown
3194 ( 3666 ) 1.62857 ( 1.97777777778 ) Kate Hodge
3195 ( 3416 ) 1.62711 ( 2.31331699346 ) Johnathon Schaech
3196 ( 1187 ) 1.62648 ( 3.5 ) Carmelita Geraghty
3197 ( 2002 ) 1.62645 ( 3.19230769231 ) Gaïl Morel
3198 ( 440 ) 1.62583 ( 3.81034482759 ) Jim Lovell
3199 ( 2768 ) 1.62542 ( 2.81 ) Frederick Stafford
3200 ( 3513 ) 1.62447 ( 2.20796460177 ) Griffin Dunne
3201 ( 1948 ) 1.62359 ( 3.21428571429 ) Zeljko Ivanek
3202 ( 3991 ) 1.62329 ( 0 ) Louise Smith
3203 ( 548 ) 1.62241 ( 3.75714285714 ) Janet Gaynor
3204 ( 2439 ) 1.62229 ( 3.0 ) Mildred Clinton
3205 ( 3723 ) 1.62119 ( 1.8 ) Ben Webster
3206 ( 2421 ) 1.61974 ( 3.0 ) Charles Love
3207 ( 3134 ) 1.61968 ( 2.55882352941 ) Michael Carmine
3208 ( 2329 ) 1.61706 ( 3.05555555556 ) Toby
3209 ( 2935 ) 1.6163 ( 2.71428571429 ) Alan Rickman
3210 ( 3931 ) 1.61589 ( 0 ) Patrick Achdjian
3211 ( 3806 ) 1.61578 ( 1.55913978495 ) Dick Martin
3212 ( 978 ) 1.61524 ( 3.59090909091 ) Y_z» Kayama
3213 (

3457 ( 2220 ) 1.38725 ( 3.1 ) Patrick Bauchau
3458 ( 2879 ) 1.38715 ( 2.75 ) Max Cullen
3459 ( 2476 ) 1.38685 ( 3.0 ) Elina L_wensohn
3460 ( 2479 ) 1.38496 ( 3.0 ) Willis Draffen Jr.
3461 ( 3207 ) 1.38432 ( 2.5 ) Bill McGhee
3462 ( 2072 ) 1.38387 ( 3.16666666667 ) Christian Campbell
3463 ( 3687 ) 1.38366 ( 1.90361445783 ) Candace Hutson
3464 ( 3650 ) 1.38317 ( 2.0 ) William R. Moses
3465 ( 208 ) 1.3829 ( 3.9375 ) Fedja van Hut
3466 ( 3805 ) 1.38055 ( 1.5612244898 ) Diane Ladd
3467 ( 3099 ) 1.38002 ( 2.58823529412 ) Rudolph Valentino
3468 ( 3926 ) 1.37765 ( 0 ) Jim Brown
3469 ( 2917 ) 1.37563 ( 2.72340425532 ) Emil Pinnock
3470 ( 3734 ) 1.37434 ( 1.77419354839 ) Dawna Wightman
3471 ( 3718 ) 1.37376 ( 1.8 ) BD Wong
3472 ( 3002 ) 1.37191 ( 2.66666666667 ) Eloise Eonnet
3473 ( 1759 ) 1.37135 ( 3.29032258065 ) Pascal Rollin
3474 ( 2227 ) 1.37134 ( 3.1 ) Jeremy James Kissner
3475 ( 3789 ) 1.37043 ( 1.625 ) Don Stark
3476 ( 2874 ) 1.37018 ( 2.75 ) Dagmar Lassander
3477 ( 3606 ) 1.36906 ( 2.0

3644 ( 2037 ) 1.19161 ( 3.17647058824 ) Nic Ratner
3645 ( 2354 ) 1.19094 ( 3.04166666667 ) Kathleen Robertson
3646 ( 2206 ) 1.18983 ( 3.10714285714 ) George Burns
3647 ( 2060 ) 1.1896 ( 3.16666666667 ) Steven Page
3648 ( 3366 ) 1.18771 ( 2.36666666667 ) Tim Matheson
3649 ( 3206 ) 1.18508 ( 2.5 ) Josh Charles
3650 ( 1216 ) 1.18462 ( 3.5 ) Matt George
3651 ( 3674 ) 1.18418 ( 1.9375 ) Danny Hoch
3652 ( 2416 ) 1.18415 ( 3.0 ) End' Kids The 'Dead
3653 ( 136 ) 1.18339 ( 4.0 ) Emily Hubley
3654 ( 1488 ) 1.18334 ( 3.40625 ) Morgane Maugran
3655 ( 1846 ) 1.1798 ( 3.25 ) Sophie Guillemin
3656 ( 1188 ) 1.17891 ( 3.5 ) Demin Bichir
3657 ( 798 ) 1.17725 ( 3.65517241379 ) Phoebe Brand
3658 ( 2793 ) 1.17637 ( 2.79464285714 ) James Le Gros
3659 ( 913 ) 1.17506 ( 3.6170212766 ) Barbara Steele
3660 ( 1607 ) 1.17442 ( 3.35714285714 ) Franoise Blanchard
3661 ( 3724 ) 1.174 ( 1.79285714286 ) Terence Knox
3662 ( 2876 ) 1.17376 ( 2.75 ) Azzedine Ala´a
3663 ( 3855 ) 1.17286 ( 1.34545454545 ) Phil Fondacaro


3852 ( 3698 ) 0.888588 ( 1.875 ) Daniel McVicar
3853 ( 3754 ) 0.888185 ( 1.75 ) Arturo Cmara
3854 ( 114 ) 0.88765 ( 4.0 ) Carma Hinton
3855 ( 3885 ) 0.886851 ( 0.823529411765 ) Jane Wheeler
3856 ( 2873 ) 0.886452 ( 2.75 ) Dominique Sanda
3857 ( 1950 ) 0.885893 ( 3.21428571429 ) Ella Raines
3858 ( 2798 ) 0.885842 ( 2.79166666667 ) Paolo Rotondo
3859 ( 3226 ) 0.876064 ( 2.5 ) Will Arnett
3860 ( 2230 ) 0.873966 ( 3.1 ) Bronia Wheeler
3861 ( 3233 ) 0.873876 ( 2.5 ) William S. Burroughs
3862 ( 2912 ) 0.872808 ( 2.72727272727 ) Sean Pertwee
3863 ( 2223 ) 0.871097 ( 3.1 ) Michael Caloz
3864 ( 3647 ) 0.868339 ( 2.0 ) Aidan Gillen
3865 ( 2364 ) 0.868116 ( 3.03571428571 ) Shaghayeh Djodat
3866 ( 3 ) 0.867957 ( 4.5 ) Bernard Hill
3867 ( 3448 ) 0.867818 ( 2.27777777778 ) Agnes Moorehead
3868 ( 3217 ) 0.867386 ( 2.5 ) Chief Thundercloud
3869 ( 1723 ) 0.864842 ( 3.30701754386 ) Mac Davis
3870 ( 2035 ) 0.862328 ( 3.17647058824 ) Brian Russell
3871 ( 3738 ) 0.861646 ( 1.76923076923 ) Crystal Bernard


In [17]:
id_2_type = {}
for i in range(50):
    id_2_type[i] = i
analysis(t_dict, _t_result, t_shape, id_2_type, 'topic_rank')

0 ( 0 ) 3.68765 ( 3.65875441627 ) 3
1 ( 2 ) 3.62873 ( 3.61198817312 ) 7
2 ( 1 ) 3.62579 ( 3.62016275094 ) 6
3 ( 3 ) 3.62189 ( 3.60498796122 ) 10
4 ( 4 ) 3.61249 ( 3.59444133452 ) 15
5 ( 5 ) 3.59695 ( 3.54289592946 ) 0
6 ( 13 ) 3.58636 ( 3.48962852465 ) 22
7 ( 15 ) 3.57582 ( 3.48257231164 ) 16
8 ( 8 ) 3.56048 ( 3.51636707123 ) 38
9 ( 6 ) 3.55249 ( 3.52507226252 ) 27
10 ( 10 ) 3.54502 ( 3.50430072036 ) 18
11 ( 9 ) 3.5419 ( 3.51398442228 ) 48
12 ( 7 ) 3.54107 ( 3.51724308633 ) 14
13 ( 12 ) 3.53447 ( 3.49909441037 ) 40
14 ( 11 ) 3.52455 ( 3.50148683062 ) 42
15 ( 14 ) 3.50983 ( 3.48658804382 ) 25
16 ( 16 ) 3.50755 ( 3.47130758596 ) 34
17 ( 17 ) 3.50007 ( 3.46878029017 ) 46
18 ( 18 ) 3.49709 ( 3.46518163542 ) 47
19 ( 19 ) 3.49442 ( 3.44804601013 ) 35
20 ( 22 ) 3.49343 ( 3.4293268084 ) 12
21 ( 20 ) 3.47187 ( 3.44270847159 ) 30
22 ( 21 ) 3.46457 ( 3.44139910279 ) 24
23 ( 28 ) 3.44272 ( 3.39397985755 ) 49
24 ( 24 ) 3.44098 ( 3.40847581072 ) 45
25 ( 27 ) 3.44032 ( 3.39864259906 ) 41
26 ( 23 ) 3.